Approximation to continuous POMDP implemented using `github.com/JuliaPOMDP/QuickPOMDPs.jl` and solved using POMCP with reference to tutorial [here](https://htmlview.glitch.me/?https://github.com/JuliaAcademy/Decision-Making-Under-Uncertainty/blob/master/html/4-Approximate-Methods.jl.html).

In [1]:
using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPPolicies, Parameters, Random, Plots, LinearAlgebra, Serialization
using POMDPTools, BasicPOMCP, D3Trees, GridInterpolations, POMCPOW, POMDPModels, Combinatorics, Dates, CSV, ParticleFilters

In [2]:
expID = Dates.format(Dates.now(), "yymd_HHMMS")

"221129_15560"

In [3]:
function log(s::String)
    s_time = Dates.format(Dates.now(), "HH:MM:SS\t")*s*"\n"
    open("./logs/"*expID*".txt", "a") do file
        write(file, s_time)
    end
    print(s_time)
end

log (generic function with 1 method)

In [4]:
log("Running experiment with ID "*expID)

15:56:00	Running experiment with ID 221129_15560


# Define Problem

In [70]:
@with_kw struct MyParameters
    N::Int = 3         # size of item set
    K::Int = 3         # size of arm set
    M::Int = 3         # size of beta set
    y::Real = 0.9     # discount factor
    umax::Real = 10    # max utility
    u_grain:: Int = 3  # granularity of utility approximation
    d_grain:: Int = 3  # granularity of arm distribution approximation
    beta:: Array{Float64} = [0, 0.01, 50.0]  # teacher beta values
end

params = MyParameters()
log(string(params))

13:58:52	MyParameters
  N: Int64 3
  K: Int64 3
  M: Int64 3
  y: Float64 0.9
  umax: Int64 10
  u_grain: Int64 3
  d_grain: Int64 3
  beta: Array{Float64}((3,)) [0.0, 0.01, 50.0]



# Create POMDP

In [61]:
struct State
    u::Array{Float64}         # list of N utility values for N items
    d::Array{Array{Float64}}  # list of K arm distributions, each assigning probabilities to N items
    b::Array{Float64}         # list of M beta values
end

In [71]:
# space of utility functions
@time begin
    umin = 0
    grid_coor = fill(range(umin,params.umax,length=params.u_grain), params.N)
    U = RectangleGrid(grid_coor...)
end

@assert length(U[1]) == params.N
log("generated "*string(length(U))*" utilities (each length "*string(length(U[1]))*" items)")

  0.000044 seconds (33 allocations: 2.969 KiB)
13:58:57	generated 27 utilities (each length 3 items)


In [72]:
function generate_probability_distributions(N::Int, coor::Array{Float64}, S::Float64=1.0)
    if S == 0
        return [[0. for _ in 1:N]]
    end
    if N == 1
        return [[float(S)]]
    end
    out = []
    range = coor[1:findall(x->isapprox(x,S,atol=1e-15), coor)[1]]
    for k in range
        subsolution = generate_probability_distributions(N-1, coor, S-k)
        for lst in subsolution
            if typeof(lst[1]) != Float64
                log("ERROR: lst "*string(lst)*" has type "*string(typeof(lst[1]))*". Must be Float64.")
            end
            prepend!(lst, float(k))
        end
        out = vcat(out, subsolution)
    end
    return out
end

generate_probability_distributions (generic function with 2 methods)

In [73]:
# space of arm distributions
@time begin
    coor = collect(range(0.,1.,length=params.d_grain))    
    simplex_list = generate_probability_distributions(params.N, coor)
    D_tuples = vec(collect(Base.product(fill(simplex_list, params.K)...)))
    D = [collect(d) for d in D_tuples]
end

@assert length(D[1]) == params.K
@assert length(D[1][1]) == params.N
log(string("generated "*string(length(D))*" arm distribution sets (each shape "*string(length(D[1]))*" arms x "*string(length(D[1][1]))*" items)"))
    
    

  0.089307 seconds (554.50 k allocations: 26.370 MiB, 99.42% compilation time)
13:59:00	generated 216 arm distribution sets (each shape 3 arms x 3 items)


In [74]:
# beta values
B = [params.beta]

# each beta value set must be length M
@assert length(B[1]) == params.M
log(string("generated "*string(length(B))*" beta value sets (each length "*string(length(B[1]))*" teachers)"))

13:59:02	generated 1 beta value sets (each length 3 teachers)


In [75]:
# State space
@time begin     
    S = [[State(u,d,b) for u in U, d in D, b in B]...,]
end

log("generated "*string(length(S))*" states")

  0.030977 seconds (116.06 k allocations: 5.993 MiB, 95.63% compilation time)
13:59:03	generated 5832 states


In [69]:
function interesting(s::State)
    e = [dot(s.u, di) for di in s.d]
    order = e[1] > e[2] && e[2] >= e[3] 
    stoch = all([s.d[i][j]!=1. for i in 1:3, j in 1:3])
    return order && stoch
end

function delta_similar(s::State, delta::Float64)
    e = [dot(s.u, di) for di in s.d]
    return abs(e[1] - e[2]) > delta && abs(e[1] - e[3]) > delta && abs(e[3] - e[2]) > delta
end


# indices = []
# for i in 1:length(S)
#     if interesting(S[i])
#         println("index ", i)
#         println(S[i])
#         push!(indices, i)
#     end
# end

delta = 0.001
indices = []
for i in 1:length(S)
    if delta_similar(S[i], delta)
#         println("index ", i)
#         println(S[i])
        push!(indices, i)
    end
end

length(indices)/length(S)

0.6997913284430807

In [37]:
s = S[942]

arm_u = [dot(s.u, d) for d in s.d]

3-element Vector{Float64}:
 6.1111111111111125
 8.333333333333334
 8.333333333333334

In [42]:
function good_state(s::State)
    # max u is <10, min u is >0, other u is in between
    u_check = (maximum(s.u) == 10) && (minimum(s.u) == 0) && (length(s.u) == length(unique(s.u)))
    
    # all d are lotteries
    d_check = (maximum(s.d[1]) < 1.) && (maximum(s.d[2]) < 1.) && (maximum(s.d[3]) < 1.)
    
    # all d are unique
    d_unique = (s.d[1] != s.d[2]) && (s.d[2] != s.d[3]) && (s.d[1] != s.d[3])
    
    # max expected arm U >= 2*other arm U
    arm_u = [dot(s.u, d) for d in s.d]
    u_gap = (arm_u[1] >= 2* arm_u[2]) && (arm_u[2] >= arm_u[3])
    
    return u_check && d_check && d_unique && u_gap
end

find = 5
found = 0
for i in 1:length(S)
    if good_state(S[i])
        println("\nState ", i, ": ")
        println(S[i])
        found += 1
        if found == find
            break
        end
    end
end


State 289506: 
State([10.0, 1.6666666666666667, 0.0], Array{Float64}[[0.0, 0.6666666666666666, 0.33333333333333337], [0.0, 0.3333333333333333, 0.6666666666666667], [0.0, 0.16666666666666666, 0.8333333333333334]], [0.0, 0.01, 50.0])

State 289513: 
State([10.0, 3.3333333333333335, 0.0], Array{Float64}[[0.0, 0.6666666666666666, 0.33333333333333337], [0.0, 0.3333333333333333, 0.6666666666666667], [0.0, 0.16666666666666666, 0.8333333333333334]], [0.0, 0.01, 50.0])

State 289520: 
State([10.0, 5.0, 0.0], Array{Float64}[[0.0, 0.6666666666666666, 0.33333333333333337], [0.0, 0.3333333333333333, 0.6666666666666667], [0.0, 0.16666666666666666, 0.8333333333333334]], [0.0, 0.01, 50.0])

State 289527: 
State([10.0, 6.666666666666667, 0.0], Array{Float64}[[0.0, 0.6666666666666666, 0.33333333333333337], [0.0, 0.3333333333333333, 0.6666666666666667], [0.0, 0.16666666666666666, 0.8333333333333334]], [0.0, 0.01, 50.0])

State 289534: 
State([10.0, 8.333333333333334, 0.0], Array{Float64}[[0.0, 0.6666666

In [44]:
function print_state(s::State)
    println("\t u: ", s.u)
    println("\t d1: ", s.d[1], "\t (exp val ", dot(s.u, s.d[1]), ")")
    println("\t d1: ", s.d[2], "\t (exp val ", dot(s.u, s.d[2]), ")")
    println("\t d1: ", s.d[3], "\t (exp val ", dot(s.u, s.d[3]), ")")
end

state_IDs = [289506, 289513, 289520, 289527, 289534]

for ID in state_IDs
    println("\nState ", ID, ":")
    print_state(S[ID])
end


State 289506:
	 u: [10.0, 1.6666666666666667, 0.0]
	 d1: [0.0, 0.6666666666666666, 0.33333333333333337]	 (exp val 1.1111111111111112)
	 d1: [0.0, 0.3333333333333333, 0.6666666666666667]	 (exp val 0.5555555555555556)
	 d1: [0.0, 0.16666666666666666, 0.8333333333333334]	 (exp val 0.2777777777777778)

State 289513:
	 u: [10.0, 3.3333333333333335, 0.0]
	 d1: [0.0, 0.6666666666666666, 0.33333333333333337]	 (exp val 2.2222222222222223)
	 d1: [0.0, 0.3333333333333333, 0.6666666666666667]	 (exp val 1.1111111111111112)
	 d1: [0.0, 0.16666666666666666, 0.8333333333333334]	 (exp val 0.5555555555555556)

State 289520:
	 u: [10.0, 5.0, 0.0]
	 d1: [0.0, 0.6666666666666666, 0.33333333333333337]	 (exp val 3.333333333333333)
	 d1: [0.0, 0.3333333333333333, 0.6666666666666667]	 (exp val 1.6666666666666665)
	 d1: [0.0, 0.16666666666666666, 0.8333333333333334]	 (exp val 0.8333333333333333)

State 289527:
	 u: [10.0, 6.666666666666667, 0.0]
	 d1: [0.0, 0.6666666666666666, 0.33333333333333337]	 (exp val 4.

In [31]:
# find a state where r(C1) == 10, r(C2) == 4, r(C3) == 0
# u = 10, 8, 2
# d1 = 100
# d2 = 010
# ..

function valid_u(s)
    u = s.u
    if u[1] == u[2] || u[1] == u[3] || u[2] == u[3] 
        return false
    end
    return true
end

function valid_d(s)
    for d in s.d
        for p in d
            if p == 0
                return false
            end
        end
    end
    return true
end

exists = false
for i in 1:length(S)
    s = S[i]
    r1 = dot(s.u, s.d[1])
    r2 = dot(s.u, s.d[2])
    r3 = dot(s.u, s.d[3])
    
    if r1 >= 2*r2 && r2 > r3
        if valid_d(s) && valid_u(s)
            println(i, "\t", s)
            exists = true
        end
    end
end
if !exists
    println("none")
end

2243573	State([3.3333333333333335, 1.6666666666666667, 0.0], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2243574	State([5.0, 1.6666666666666667, 0.0], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2243575	State([6.666666666666667, 1.6666666666666667, 0.0], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2243576	State([8.333333333333334, 1.6666666666666667, 0.0], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666

2244611	State([6.666666666666667, 3.3333333333333335, 0.0], Array{Float64}[[0.5, 0.3333333333333333, 0.16666666666666669], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2244612	State([8.333333333333334, 3.3333333333333335, 0.0], Array{Float64}[[0.5, 0.3333333333333333, 0.16666666666666669], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2244613	State([10.0, 3.3333333333333335, 0.0], Array{Float64}[[0.5, 0.3333333333333333, 0.16666666666666669], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2244620	State([10.0, 5.0, 0.0], Array{Float64}[[0.5, 0.3333333333333333, 0.16666666666666669], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2245631	State([3.3333333333333335, 1.6666

2246670	State([8.333333333333334, 3.3333333333333335, 0.0], Array{Float64}[[0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2246671	State([10.0, 3.3333333333333335, 0.0], Array{Float64}[[0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2246676	State([6.666666666666667, 5.0, 0.0], Array{Float64}[[0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2246677	State([8.333333333333334, 5.0, 0.0], Array{Float64}[[0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17], [0.16666666666666666, 0.3333333333333333, 0.5], [0.16666666666666666, 0.16666666666666666,

2256274	State([8.333333333333334, 3.3333333333333335, 0.0], Array{Float64}[[0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17], [0.16666666666666666, 0.5, 0.33333333333333337], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2256275	State([10.0, 3.3333333333333335, 0.0], Array{Float64}[[0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17], [0.16666666666666666, 0.5, 0.33333333333333337], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2256281	State([8.333333333333334, 5.0, 0.0], Array{Float64}[[0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17], [0.16666666666666666, 0.5, 0.33333333333333337], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2256282	State([10.0, 5.0, 0.0], Array{Float64}[[0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17], [0.16666666666666666, 0.5, 0.33333333333333337], [0.16666666666666666, 0.16666666666666666, 0.666666

2291629	State([5.0, 10.0, 0.0], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.3333333333333333, 0.16666666666666666, 0.5000000000000001], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2291630	State([6.666666666666667, 10.0, 0.0], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.3333333333333333, 0.16666666666666666, 0.5000000000000001], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2291631	State([8.333333333333334, 10.0, 0.0], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.3333333333333333, 0.16666666666666666, 0.5000000000000001], [0.16666666666666666, 0.16666666666666666, 0.6666666666666667]], [0.01, 10.0])
2291670	State([3.3333333333333335, 8.333333333333334, 1.6666666666666667], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.3333333333333333, 0.16666666666666666, 0.5000000

...

7216616	State([0.0, 10.0, 6.666666666666667], Array{Float64}[[0.16666666666666666, 0.6666666666666666, 0.16666666666666674], [0.6666666666666666, 0.16666666666666666, 0.1666666666666667], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7216617	State([1.6666666666666667, 10.0, 6.666666666666667], Array{Float64}[[0.16666666666666666, 0.6666666666666666, 0.16666666666666674], [0.6666666666666666, 0.16666666666666666, 0.1666666666666667], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7216644	State([0.0, 5.0, 8.333333333333334], Array{Float64}[[0.16666666666666666, 0.6666666666666666, 0.16666666666666674], [0.6666666666666666, 0.16666666666666666, 0.1666666666666667], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7216651	State([0.0, 6.666666666666667, 8.333333333333334], Array{Float64}[[0.16666666666666666, 0.6666666666666666, 0.16666666666666674], [0.6666666666666666, 0.166666666666

7217932	State([0.0, 8.333333333333334, 5.0], Array{Float64}[[0.3333333333333333, 0.3333333333333333, 0.3333333333333334], [0.6666666666666666, 0.16666666666666666, 0.1666666666666667], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7217939	State([0.0, 10.0, 5.0], Array{Float64}[[0.3333333333333333, 0.3333333333333333, 0.3333333333333334], [0.6666666666666666, 0.16666666666666666, 0.1666666666666667], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7217953	State([0.0, 1.6666666666666667, 6.666666666666667], Array{Float64}[[0.3333333333333333, 0.3333333333333333, 0.3333333333333334], [0.6666666666666666, 0.16666666666666666, 0.1666666666666667], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7217960	State([0.0, 3.3333333333333335, 6.666666666666667], Array{Float64}[[0.3333333333333333, 0.3333333333333333, 0.3333333333333334], [0.6666666666666666, 0.16666666666666666, 0.1666666666666

7218577	State([1.6666666666666667, 10.0, 3.3333333333333335], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.6666666666666666, 0.16666666666666666, 0.1666666666666667], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7218611	State([0.0, 6.666666666666667, 5.0], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.6666666666666666, 0.16666666666666666, 0.1666666666666667], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7218618	State([0.0, 8.333333333333334, 5.0], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.6666666666666666, 0.16666666666666666, 0.1666666666666667], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7218625	State([0.0, 10.0, 5.0], Array{Float64}[[0.3333333333333333, 0.6666666666666666, 1.1102230246251565e-16], [0.6666666666666666, 0.16666666666666666, 0.1666666666

Excessive output truncated after 524298 bytes.

[0.01, 10.0])
7225401	State([0.0, 1.6666666666666667, 3.3333333333333335], Array{Float64}[[0.16666666666666666, 0.3333333333333333, 0.5], [0.6666666666666666, 0.3333333333333333, 5.551115123125783e-17], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7225450	State([0.0, 1.6666666666666667, 5.0], Array{Float64}[[0.16666666666666666, 0.3333333333333333, 0.5], [0.6666666666666666, 0.3333333333333333, 5.551115123125783e-17], [0.8333333333333334, 0.16666666666666666, -2.7755575615628914e-17]], [0.01, 10.0])
7225457	State([0.0, 3.3333333333333335, 5.0], Array{Float64}[[0.16666666666666666, 

In [16]:
# Action space - actions are arm choices (K) or beta selections (M)
struct Action
    name::String      # valid names are {B,C} + index
    isBeta::Bool      # true if 'B' action, false if 'C' action
    index::Integer    # index of beta (if 'B' action) or arm choice (if 'C' action)
end

A = Array{Action}(undef, params.K+params.M)
for i in 1:params.K+params.M
    if i <= params.K
        A[i] = Action("C"*string(i), false, i)
    else
        A[i] = Action("B"*string(i-params.K), true, i-params.K)
    end
end
log("generated "*string(length(A))*" actions")

11:24:38	generated 5 actions


In [17]:
# Transition function
function T(s::State, a::Action)
    return SparseCat([s], [1.0])    # categorical distribution
end
log("generated transition function")

11:24:38	generated transition function


In [18]:
# Reward function
function R(s::State, a::Action)
    # if beta selected, return 0
    if a.isBeta
        return 0
    # if arm pulled, return that arm's avg utility
    else
        utilities = s.u
        arm_dist = s.d[a.index]
        return dot(utilities, arm_dist)
    end
end
log("generated reward function")

11:24:39	generated reward function


In [19]:
# item space
I = 1:params.N

# preference space
struct Preference
    i0::Int    # first item to compare, in {1,2,...,N}
    i1::Int    # second item to compare, in {1,2,...,N}
    label::Int # feedback label, in {0,1}
end

P = [[Preference(i0,i1,label) for i0 in I, i1 in I, label in [0,1]]...,]

# observation space
struct Observation
    isItem::Bool    # true if item returned, false otherwise
    i::Int          # item, if item returned
    p::Preference   # preference, if preference returned
end

invalid_i = -1
invalid_p = Preference(-1,-1,-1)
I_obs = [Observation(true, i, invalid_p) for i in I]
P_obs = [Observation(false, invalid_i, p) for p in P]
omega = union(I_obs, P_obs)

log("generated "*string(length(omega))*" observations")

11:24:39	generated 21 observations


In [20]:
# unnormalized query profile (likelihood of querying 1,1; 2,1; 3,1; ... ; N,1; 1,2; 2,2; ... ; N,N)
Q = ones(params.N*params.N)

# preference probability (expected preference, or probability that preference=1)
function Pr(p::Preference, s::State, b::Float64)
    prob_pref_1 = exp(Float64(b)*s.u[p.i1])/(exp(Float64(b)*s.u[p.i1])+exp(Float64(b)*s.u[p.i0]))
    if p.label == 1
        return prob_pref_1
    else
        return 1.0-prob_pref_1
    end
end

Pr (generic function with 1 method)

In [21]:
function O(s::State, a::Action, sp::State)
    # if B action, obs in P_obs
    if a.isBeta
        prob_of_pref = [Pr(o.p, s, s.b[a.index]) for o in P_obs]
        prob_of_query = vcat(Q,Q)   # doubled because each query appears once for each label
        
        # weight by querying profile to get dist
        dist = [prob_of_pref[i]*prob_of_query[i] for i in 1:length(prob_of_pref)]
        normalized_dist = dist/sum(dist)        
        return SparseCat(P_obs, normalized_dist)
    # if C action, obs in I_obs
    else
        return SparseCat(I_obs, s.d[a.index])
    end
end

log("generated observation function")

11:24:41	generated observation function


In [22]:
@time begin
    
    # define POMDP
    abstract type MyPOMDP <: POMDP{State, Action, Observation} end
    pomdp = QuickPOMDP(MyPOMDP,
        states       = S,
        actions      = A,
        observations = omega,
        transition   = T,
        observation  = O,
        reward       = R,
        discount     = params.y,
        initialstate = S);

end

log("created POMDP")

  0.741701 seconds (2.31 M allocations: 256.848 MiB, 10.81% gc time, 66.64% compilation time)
11:24:43	created POMDP


In [23]:
rollout = true

if rollout
    policy = RandomPolicy(pomdp)

    show_state = true
    for (s,a,r,o) in stepthrough(pomdp, policy, "s,a,r,o", max_steps=3)
        if show_state
            @show s
            println("")
            show_state = false
        end
        @show a
        @show r
        @show o
        println()
    end
end

s = State([2.0, 0.0, 6.0], Array{Float64}[[0.4, 0.6, 0.0], [0.4, 0.6, 0.0], [0.4, 0.6, 0.0]], [0.01, 10.0])

a = Action("C2", false, 2)
r = 0.8
o = Observation(true, 1, Preference(-1, -1, -1))

a = Action("C1", false, 1)
r = 0.8
o = Observation(true, 1, Preference(-1, -1, -1))

a = Action("B2", true, 2)
r = 0
o = Observation(false, -1, Preference(1, 2, 0))



# Solve POMDP

In [24]:
@time begin
    solver = POMCPOWSolver()
    planner = solve(solver, pomdp);
end
log("solved POMDP")

open("./policies/"*expID*"_policy.txt", "w") do file
    serialize(file, planner)
end

log("saved policy to "*"./policies/"*expID*"_policy.txt")

  0.033817 seconds (43.64 k allocations: 2.558 MiB, 98.61% compilation time)
11:24:53	solved POMDP
11:25:02	saved policy to ./policies/22105_112423_policy.txt


In [25]:
action(planner, Uniform(S))

Action("C3", false, 3)

In [26]:
rollout = true

if rollout
    show_state = true
    for (s,a,r,o) in stepthrough(pomdp, planner, "s,a,r,o", max_steps=3)
        if show_state
            @show s
            println("")
            show_state = false
        end
        @show a
        @show r
        @show o
        println()
    end
end

s = State([2.0, 4.0, 10.0], Array{Float64}[[0.4, 0.6, 0.0], [0.6, 0.0, 0.4], [0.2, 0.6, 0.20000000000000007]], [0.01, 10.0])

a = Action("C3", false, 3)
r = 4.800000000000001
o = Observation(true, 2, Preference(-1, -1, -1))

a = Action("C3", false, 3)
r = 4.800000000000001
o = Observation(true, 2, Preference(-1, -1, -1))

a = Action("C1", false, 1)
r = 3.2
o = Observation(true, 2, Preference(-1, -1, -1))



# Evaluate Solution

In [ ]:
# optionally read planner from file
read_in_planner = false
if read_in_planner
    planner = deserialize(open("./policies/"*expID*"_policy.txt", "r"))
end

In [74]:
aₚ, info = action_info(planner, initialstate(pomdp), tree_in_info=true); aₚ
tree = D3Tree(info[:tree], init_expand=3)

o: <root>
N: 1000      

├──a: Action("C3", false, 3)
│  N: 3       V: 283       
│  ├──o: Observation(true, 1, Preference(-1, -1, -1))
│  │  N: 1         

│  │  └──a: Action("C1", false, 1)
│  │     N: 1       V: 232        (1 children)
│  └──o: Observation(true, 4, Preference(-1, -1, -1))
│     N: 0         

├──a: Action("C1", false, 1)
│  N: 1       V: 224       
│  └──o: Observation(true, 4, Preference(-1, -1, -1))
│     N: 0         

├──a: Action("B2", true, 2)
│  N: 994     V: 309       
│  ├──o: Observation(false, -1, Preference(1, 1, 1))
│  │  N: 34        

│  │  ├──a: Action("C3", false, 3)
│  │  │  N: 3       V: 307        (2 children)
│  │  ├──a: Action("B2", true, 2)
│  │  │  N: 2       V: 289        (2 children)
│  │  ├──a: Action("B1", true, 1)
│  │  │  N: 2       V: 243        (2 children)
│  │  ├──a: Action("C2", false, 2)
│  │  │  N: 5       V: 299        (3 children)
│  │  └──a: Action("C1", false, 1)
│  │     N: 22      V: 342        (4 children)
│  ├──o: Observation(false, -1, Preference(3, 1, 0))
│  │  N: 26        

│  │  ├──a: Action("B1", true, 1)
│  │  │  N: 22      V: 350        (18 children)
│  │  ├──a: Action("C1", false, 1)
│  │  │  N: 1       V: 190        (1 children)
│  │  ├──a: Action("C3", false, 3)
│  │  │  N: 1       V: 115        (1 children)
│  │  ├──a: Action("C2", false, 2)
│  │  │  N: 1       V: 88.3       (1 children)
│  │  └──a: Action("B2", true, 2)
│  │     N: 1       V: 212        (1 children)
│  ├──o: Observation(false, -1, Preference(2, 4, 0))
│  │  N: 42        

│  │  ├──a: Action("C1", false, 1)
│  │  │  N: 2       V: 175        (2 children)
│  │  ├──a: Action("B1", true, 1)
│  │  │  N: 8       V: 290        (8 children)
│  │  ├──a: Action("C2", false, 2)
│  │  │  N: 1       V: 102        (1 children)
│  │  ├──a: Action("B2", true, 2)
│  │  │  N: 30      V: 291        (21 children)
│  │  └──a: Action("C3", false, 3)
│  │     N: 1       V: 246        (1 children)
│  ├──o: Observation(false, -1, Preference(4, 1, 1))
│  │  N: 51        

│  │  ├──a: Action("C3", false, 3)
│  │  │  N: 2       V: 302        (1 children)
│  │  ├──a: Action("C1", false, 1)
│  │  │  N: 46      V: 359        (4 children)
│  │  ├──a: Action("B1", true, 1)
│  │  │  N: 1       V: 313        (1 children)
│  │  ├──a: Action("B2", true, 2)
│  │  │  N: 1       V: 313        (1 children)
│  │  └──a: Action("C2", false, 2)
│  │     N: 1       V: 269        (1 children)
│  ├──o: Observation(false, -1, Preference(2, 3, 0))
│  │  N: 35        

│  │  ├──a: Action("B2", true, 2)
│  │  │  N: 1       V: 414        (1 children)
│  │  ├──a: Action("B1", true, 1)
│  │  │  N: 8       V: 391        (6 children)
│  │  ├──a: Action("C3", false, 3)
│  │  │  N: 11      V: 400        (4 children)
│  │  ├──a: Action("C1", false, 1)
│  │  │  N: 14      V: 417        (4 children)
│  │  └──a: Action("C2", false, 2)
│  │     N: 1       V: 0          (1 children)
│  ├──o: Observation(false, -1, Preference(4, 4, 0))
│  │  N: 38        

│  │  ├──a: Action("C2", false, 2)
│  │  │  N: 1       V: 116        (1 children)
│  │  ├──a: Action("C3", false, 3)
│  │  │  N: 1       V: 116        (1 children)
│  │  ├──a: Action("C1", false, 1)
│  │  │  N: 34      V: 167        (4 children)
│  │  ├──a: Action("B2", true, 2)
│  │  │  N: 1       V: 0          (1 children)
│  │  └──a: Action("B1", true, 1)
│  │     N: 1       V: 0          (1 children)
│  ├──o: Observation(false, -1, Preference(4, 1, 0))
│  │  N: 13        

│  │  ├──a: Action("C1", false, 1)
│  │  │  N: 2       V: 80.7       (2 children)
│  │  ├──a: Action("C2", false, 2)
│  │  │  N: 1       V: 53.1       (1 children)
│  │  ├──a: Action("B2", true, 2)
│  │  │  N: 8       V: 172        (8 children)
│  │  └──a: Action("B1", true, 1)
│  │     N: 2       V: 70.6       (2 children)
│  ├──o: Observation(false, -1, Preference(2, 3, 1))
│  │  N: 40        

│  │  ├──a: Action("B2", true, 2)
│  │  │  N: 32      V: 210        (21 children)
│  │  ├──a: Action("C3", false, 3)
│  │  │  N: 5

In [222]:
steps = 1
iters = 2
is = State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0])
# sim = StepSimulator("a", max_steps=steps)
# upd = 
# prior = 

# trying to use
# function simulate(sim::StepSimulator, pomdp::POMDP, policy::Policy, bu::Updater, dist::Any, is=rand(sim.rng, initialstate(pomdp)))
# iterator = simulate(sim, pomdp, planner, upd, prior, init_state)
# iterator = stepthrough(pomdp, planner, init_state)
# iterator = simulate(sim, pomdp, planner, upd, prior, init_state)
# for (a) in iterator
#     println(a)
# end

iterator = stepthrough(pomdp, planner, updater(planner), Uniform(S), is, "s,a,o,r", max_steps=10)
action(planner, prior)
for (s, a, o, r) in iterator
    @show s
    @show a
end

s = State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0])
a = Action("C2", false, 2)
s = State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0])
a = Action("B2", true, 2)
s = State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0])
a = Action("C1", false, 1)
s = State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0])
a = Action("C2", false, 2)
s = State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0])
a = Action("C1", false, 1)
s = State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0])
a = Action("C3", false, 3)
s = State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0])
a = Action("C2", false, 2)
s = State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 

In [27]:
steps = 100
iters = 2

initial_states = [State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0]) for i in 1:iters]
POMCPOW_R = Array{Float64}(undef, iters)
beliefs = Array{Array{ParticleFilters.ParticleCollection{State}}}(undef, (iters, steps))
for iter in 1:iters
    log("running simulation "*string(iter))
    
    t = 1
    r_accum = 0.
    beliefs_iter = Array{ParticleFilters.ParticleCollection{State}}(undef, steps)
    for (s, a, o, r, b) in stepthrough(pomdp, planner, updater(planner), Uniform(S), initial_states[iter], "s,a,o,r,b", max_steps=steps)
        r_accum = r_accum + r
        beliefs_iter[t] = b
        
        if t == 1
            open("./sims/"*expID*"_run"*string(iter)*".txt", "w") do file
                write(file, string(s))
            end
            println(s)
        end
        
        if a.isBeta
            msg = "\n"*string(t)*",B,"*a.name*",(i"*string(o.p.i0)*"-i"*string(o.p.i1)*";"*string(o.p.label)*"),"*string(r)
        else
            msg = "\n"*string(t)*",C,"*a.name*",i"*string(o.i)*","*string(r)
        end

        open("./sims/"*expID*"_run"*string(iter)*".txt", "a") do file
            write(file, msg)
        end
        t = t + 1
    end
    beliefs[iter] = beliefs_iter
    POMCPOW_R[iter] = r_accum
end
    
log("ran "*string(iters)*" POMCPOW rollouts for "*string(steps)*" timesteps each")
log("POMCPOW R: "*string(POMCPOW_R))

open("./beliefs/"*expID*"_belief.txt", "w") do file
    serialize(file, beliefs)
end

log("saved beliefs to "*"./beliefs/"*expID*"_belief.txt")

11:25:32	running simulation 1
State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0])
11:25:44	running simulation 2
State([0.0, 8.0, 10.0], Array{Float64}[[0.0, 0.0, 1.0], [0.6, 0.0, 0.4], [1.0, 0.0, 0.0]], [0.01, 10.0])
11:25:56	ran 2 POMCPOW rollouts for 100 timesteps each
11:25:56	POMCPOW R: [520.0, 474.0]
11:25:57	saved beliefs to ./beliefs/22105_112423_belief.txt


In [251]:
new_beliefs = deserialize(open("./beliefs/"*"2299_123850"*"_belief.txt", "r"))

5×1000 Matrix{Array{ParticleCollection{State}}}:
 [ParticleCollection{State}(State[State([2.0, 4.0, 0.0], Array{Float64}[[0.4, 0.2, 0.4], [0.0, 0.4, 0.6], [0.0, 0.4, 0.6]], [0.01, 10.0]), State([6.0, 4.0, 8.0], Array{Float64}[[0.8, 0.2, -5.55112e-17], [0.6, 0.2, 0.2], [0.6, 0.2, 0.2]], [0.01, 10.0]), State([6.0, 8.0, 10.0], Array{Float64}[[0.4, 0.0, 0.6], [1.0, 0.0, 0.0], [0.8, 0.2, -5.55112e-17]], [0.01, 10.0]), State([6.0, 8.0, 6.0], Array{Float64}[[0.8, 0.2, -5.55112e-17], [0.2, 0.6, 0.2], [0.2, 0.4, 0.4]], [0.01, 10.0]), State([10.0, 10.0, 6.0], Array{Float64}[[0.2, 0.4, 0.4], [0.0, 1.0, 0.0], [0.4, 0.2, 0.4]], [0.01, 10.0]), State([6.0, 6.0, 4.0], Array{Float64}[[0.2, 0.2, 0.6], [0.2, 0.0, 0.8], [0.2, 0.0, 0.8]], [0.01, 10.0]), State([8.0, 0.0, 8.0], Array{Float64}[[1.0, 0.0, 0.0], [0.0, 0.2, 0.8], [0.2, 0.4, 0.4]], [0.01, 10.0]), State([8.0, 0.0, 4.0], Array{Float64}[[0.4, 0.2, 0.4], [0.0, 0.8, 0.2], [0.0, 0.8, 0.2]], [0.01, 10.0]), State([8.0, 8.0, 0.0], Array{Float64}[[0.6, 0.4

In [124]:
prior = Uniform(S)
sim = RolloutSimulator(max_steps=steps)

random_R = zeros(iters)
for iter in 1:iters
    # use the same initial states as the POMCPOW runs
    initial_state = initial_states[iter]
    up = updater(RandomPolicy(pomdp))
    result = simulate(sim, pomdp, RandomPolicy(pomdp), up, prior, initial_state)
    random_R[iter] = result
end

log("ran "*string(iters)*" random rollouts for "*string(steps)*" timesteps each")
log("Random R: "*string(random_R))


12:55:26	ran 5 random rollouts for 10 timesteps each
12:55:26	Random R: [6.990490109302424, 23.828074247609745, 38.82661357906989, 45.5265987243873, 35.815350182425696]


In [119]:
max_R = zeros(iters)

for iter in 1:iters
    # use the same initial states as the POMCPOW runs
    initial_state = initial_states[iter]
    max_R[iter] = maximum([dot(initial_state.u, initial_state.d[i]) for i in 1:params.K])*steps
end

log("Max R: "*string(max_R))

12:44:18	Max R: [1.0e6, 1.0e6, 1.0e6, 1.0e6, 1.0e6]


In [122]:
fig = plot(1:iters, [random_R,POMCPOW_R], 
    seriestype = :scatter, 
    label=["random" "POMCP"], 
    xticks = 0:1:iters,
    xlabel = "run",
    ylabel = "reward (" * string(steps) * " timesteps)",
    ylims = (0,maximum(POMCPOW_R)*1.2)
)
savefig(fig,"./plots/reward_ID"*string(expID)*"_step"*string(steps)*"_roll"*string(iters)*".png")